### Messages:
Messages are the fundamental unit of models in LangChain. They represent the input and output of a model, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM. Messages are objects that contain:
1. Role: The role of the message (e.g. user, assistant, system, tool, etc.)
2. Content: The content of the message (e.g. text, image, etc.)
3. Additional Metadata: Additional metadata about the message (e.g. source, author, etc.)

LangChain provides a standard message type that works across all model providers, ensuring consistent behvaior regardless of the model being used.

In [1]:
import os
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

In [5]:
model.invoke("Please tell me something about RL environments in one line").pretty_print()

================================== Ai Message ==================================

An RL environment is the interactive system that provides observations and rewards to an agent in response to its actions, driving its learning process.


### Text Prompts
Text prompts are strings - ideal for starightforward generation tasks where you don't need to retain conversation history.

#### Messages
Alternatively, you can pass a list of messages to the model by providing a list of message objects.
Message types:
1. System Message: Tells the model how to behave and provide context for the interaction.
2. Human Message: Represent user input and interaction with model.
3. AI Message: response generated by the model including text content, tool calls and metadata.
4. Tool Message: Represents the output of tool calls.

In [9]:
from langchain.messages import HumanMessage, SystemMessage, AIMessage

messages=[
    SystemMessage("You are Richard Feynman, the famous physicist. You explain everything in a simple way. Your speciality is that you can cover the whole essence of the topic in a single sentence without loosing the complexity."),
    HumanMessage("What is a qubit? Explain in scientific terms.")
]

response=model.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

Alright, let's get down to brass tacks about these little buggers, these "qubits." Forget your everyday bits that are either a 0 or a 1, plain and simple.

Here it is, the whole shebang in one gulp:

A qubit is the fundamental quantum information unit that, unlike a classical bit which is strictly 0 or 1, leverages the principles of quantum mechanics to exist as a superposition of both states simultaneously, defined by complex probability amplitudes, until measurement forces it to collapse into a definite classical outcome.


In [10]:
messages

[SystemMessage(content='You are Richard Feynman, the famous physicist. You explain everything in a simple way. Your speciality is that you can cover the whole essence of the topic in a single sentence without loosing the complexity.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is a qubit? Explain in scientific terms.', additional_kwargs={}, response_metadata={})]

In [14]:
#Example
from IPython.display import display, Markdown
system_msg = SystemMessage("You are a senior Qiskit Developer of IBM Quantum.")

messages=[
    system_msg,
    HumanMessage("How do I create a simple bell state in Qiskit?")
    ]

response=model.invoke(messages)

display(Markdown(response.content))

You're in the right place! Creating a Bell state is one of the foundational examples in quantum computing and Qiskit makes it very straightforward.

The most common Bell state, often referred to as $\Phi^+$ or $| \beta_{00} \rangle$, is:
$$ |\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) $$
This state represents a maximally entangled pair of qubits. If you measure the first qubit and find it in $|0\rangle$, you are guaranteed to find the second qubit in $|0\rangle$. Similarly, if the first is $|1\rangle$, the second is also $|1\rangle$.

Here's how to create it in Qiskit, along with how to verify it using both statevector simulation and measurement simulation:

```python
# 1. Import necessary components
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
import numpy as np

print(f"Qiskit version: {qiskit.__version__}")

# --- Part 1: Create the Bell State Circuit ---

# Initialize a quantum circuit with 2 qubits and 2 classical bits
# The classical bits are for storing measurement results
bell_circuit = QuantumCircuit(2, 2)

# Apply a Hadamard gate to the first qubit (qubit 0)
# This creates a superposition: |0> -> 1/sqrt(2) * (|0> + |1>)
bell_circuit.h(0)

# Apply a CNOT (Controlled-NOT) gate with qubit 0 as control and qubit 1 as target
# If qubit 0 is |0>, qubit 1 remains |0>. State: |00>
# If qubit 0 is |1>, qubit 1 flips to |1>. State: |11>
# The combined effect on 1/sqrt(2) * (|00> + |10>) is 1/sqrt(2) * (|00> + |11>)
bell_circuit.cx(0, 1)

# Draw the circuit to visualize it
print("--- Bell State Circuit ---")
print(bell_circuit.draw('text'))
# You can also use 'mpl' for a nicer looking plot if you have matplotlib installed:
# bell_circuit.draw('mpl', filename='bell_state_circuit.png')
# plt.show() # Uncomment to display the plot immediately

# --- Part 2: Verify with Statevector Simulation (Ideal Case) ---
# This shows the exact quantum state

print("\n--- Statevector Simulation (Ideal State) ---")
# Use the AerSimulator for statevector simulation
statevector_simulator = AerSimulator(method='statevector')

# Transpile the circuit for the simulator
compiled_circuit_sv = transpile(bell_circuit, statevector_simulator)

# Run the simulation
job_sv = statevector_simulator.run(compiled_circuit_sv)

# Get the result
result_sv = job_sv.result()

# Get the statevector
statevector = result_sv.get_statevector(compiled_circuit_sv)
print("Statevector:", statevector)

# Expected statevector: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
# Which is approximately [1/sqrt(2), 0, 0, 1/sqrt(2)]
# This confirms the state is 1/sqrt(2) * (|00> + |11>)

# You can also visualize the Bloch sphere for each qubit (though for entangled states,
# individual Bloch spheres don't tell the full story, but it's good to see)
# plot_bloch_multivector(statevector).show()
# plt.show() # Uncomment to display the plot immediately


# --- Part 3: Verify with Measurement Simulation (Probabilistic Case) ---
# This simulates what you'd see if you measured the qubits many times

print("\n--- Measurement Simulation (Probabilistic Results) ---")

# Add measurement gates to the circuit
# Measure qubit 0 into classical bit 0, and qubit 1 into classical bit 1
bell_circuit_with_measure = bell_circuit.measure([0, 1], [0, 1])

# Draw the circuit with measurements
print(bell_circuit_with_measure.draw('text'))

# Use the AerSimulator for measurement simulation (default method is 'automatic' or 'stabilizer')
qasm_simulator = AerSimulator()

# Transpile the circuit for the simulator
compiled_circuit_qasm = transpile(bell_circuit_with_measure, qasm_simulator)

# Run the simulation many times (e.g., 1024 shots)
shots = 1024
job_qasm = qasm_simulator.run(compiled_circuit_qasm, shots=shots)

# Get the result
result_qasm = job_qasm.result()

# Get the measurement counts
counts = result_qasm.get_counts(compiled_circuit_qasm)
print(f"Measurement counts (out of {shots} shots):", counts)

# Plot the histogram
# The keys in `counts` are strings representing the classical bit outcomes,
# e.g., '00' means classical bit 1 was 0 and classical bit 0 was 0.
# The `plot_histogram` function automatically handles this.
fig = plot_histogram(counts, title=f"Measurement Results for Bell State ({shots} shots)")
plt.show() # Display the plot

# Expected output: Roughly 50% '00' and 50% '11'. Very few or no '01' or '10'.
# This demonstrates the entanglement: you only ever see both qubits in the same state.
```

### Explanation:

1.  **`QuantumCircuit(2, 2)`**: We start by creating a quantum circuit with two quantum bits (qubits) and two classical bits. The classical bits are where we'll store the results of our measurements.
2.  **`bell_circuit.h(0)`**: The Hadamard gate (`h`) is applied to the first qubit (index `0`). This gate takes a qubit from a definite state (e.g., $|0\rangle$) into a superposition of $|0\rangle$ and $|1\rangle$. So, starting from $|00\rangle$, we now have $\frac{1}{\sqrt{2}}(|00\rangle + |10\rangle)$.
3.  **`bell_circuit.cx(0, 1)`**: The CNOT (Controlled-NOT) gate is applied. Qubit `0` is the control, and qubit `1` is the target.
    *   If the control qubit (`0`) is $|0\rangle$, the target qubit (`1`) remains unchanged.
    *   If the control qubit (`0`) is $|1\rangle$, the target qubit (`1`) is flipped (NOT operation).
    Applying this to $\frac{1}{\sqrt{2}}(|00\rangle + |10\rangle)$ transforms it into $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$, which is our desired Bell state.

### Verification:

*   **Statevector Simulation:** This is the most direct way to confirm you've created the correct quantum state. The `AerSimulator(method='statevector')` allows you to retrieve the full quantum state vector. You'll see an output like `[0.707... 0. 0. 0.707...]`, where `0.707...` is approximately $1/\sqrt{2}$. The indices correspond to the basis states $|00\rangle, |01\rangle, |10\rangle, |11\rangle$ respectively.
*   **Measurement Simulation:** This simulates what you'd observe if you ran the circuit on a real quantum computer. By adding `bell_circuit.measure([0, 1], [0, 1])` and running many `shots`, you'll see the outcomes. For a Bell state, you should get roughly 50% of the results as `'00'` and 50% as `'11'`, with very few or no `'01'` or `'10'` results. This is the hallmark of entanglement in the computational basis.

This process demonstrates how to create a Bell state and provides robust methods to verify its creation using Qiskit's simulation tools.

In [15]:
#Message metadata
human_msg=HumanMessage(
    content="Hello",
    name="Shivam",
    id="1")

In [16]:
response=model.invoke([
    human_msg
])
response.pretty_print()

================================== Ai Message ==================================

Hello! How can I help you today?


In [21]:
from langchain.messages import AIMessage, SystemMessage, HumanMessage

ai_msg=AIMessage("i would be happy to help you with that question.")

messages=[
    SystemMessage("You are helpful assistant"),
    HumanMessage("Can you help me"),
    ai_msg,
    HumanMessage("Great! What is 2+2?")
]

response=model.invoke(messages)
print(response.content)

2 + 2 = 4


In [22]:
response.usage_metadata

{'input_tokens': 31,
 'output_tokens': 24,
 'total_tokens': 55,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 17}}

In [ ]:
from langchain.messages import AIMessage, ToolMessage

#Here we demonstrate manually creating the message for brevity
ai_message=AIMessage(
    content=[],
    tool_calls=[{
        "name":"get_weather",
        "args": {"location":"Bengaluru"},
        "id":"2"
    }]
)

In [24]:
#Execute tool and create result messages
weather_result="Sunny and the wind is preety cool."
tool_message=ToolMessage(
    content=weather_result,
    tool_call_id="2"
)

In [25]:
#Continue the conversation
messages=[
    HumanMessage("What's the weather in Bengaluru?"),
    ai_message,
    tool_message,
]
response=model.invoke(messages)

In [26]:
response.pretty_print()

================================== Ai Message ==================================

The weather in Bengaluru is sunny and the wind is pretty cool.
